In [23]:
import loadDataForSKtime

loader = loadDataForSKtime.PatientTimeSeriesLoader(
    "C:/Users/emily/Documents/DissertationProject/training/training_setA_csv", ['HR', 'SBP', 'MAP', 'DBP', 'Resp', 'Temp'])
df = loader.load_data(100)
y_train, y_test = loader.split_train_test(df)


<class 'list'>
Patient 1: Dropping — constant columns found
Patient 2: Less than 46 time points — skipping.
Patient 4: Less than 46 time points — skipping.
Patient 5: Dropping — constant columns found
Patient 6: Less than 46 time points — skipping.
Patient 7: Less than 46 time points — skipping.
Patient 8: Less than 46 time points — skipping.
Patient 10: Less than 46 time points — skipping.
Patient 11: Less than 46 time points — skipping.
Patient 12: Less than 46 time points — skipping.
Patient 13: Less than 46 time points — skipping.
Patient 14: Less than 46 time points — skipping.
Patient 15: Less than 46 time points — skipping.
Patient 16: Less than 46 time points — skipping.
Patient 17: Less than 46 time points — skipping.
Patient 20: Less than 46 time points — skipping.
Patient 22: Less than 46 time points — skipping.
Patient 23: Less than 46 time points — skipping.
Patient 24: Dropping — constant columns found
Patient 25: Dropping — constant columns found
Patient 26: Less than 46

In [3]:
import loaders

loader = loaders.DataLoader(
    "C:/Users/emily/Documents/DissertationProject/training/training_setA_csv", ['HR', 'SBP', 'MAP', 'DBP', 'Resp', 'Temp'])
patients = loader.get_subset(max_patients=100, min_time_points=30, fill_type="ffill")

Patient 0: Dropping — constant columns found
Patient 1: Less than 30 time points — skipping.
Patient 3: Less than 30 time points — skipping.
Patient 4: Dropping — constant columns found
Patient 5: Less than 30 time points — skipping.
Patient 9: Less than 30 time points — skipping.
Patient 11: Less than 30 time points — skipping.
Patient 12: Dropping — constant columns found
Patient 14: Less than 30 time points — skipping.
Patient 15: Less than 30 time points — skipping.
Patient 16: Dropping — constant columns found
Patient 19: Less than 30 time points — skipping.
Patient 21: Less than 30 time points — skipping.
Patient 22: Less than 30 time points — skipping.
Patient 23: Dropping — constant columns found
Patient 24: Dropping — constant columns found
Patient 25: Less than 30 time points — skipping.
Patient 26: Less than 30 time points — skipping.
Patient 31: Dropping — constant columns found
Patient 32: Dropping — constant columns found
Patient 33: Less than 30 time points — skipping.
P

In [4]:
patients = loader.get_subset(max_patients=100, min_time_points=30, fill_type="ffill")

Patient 0: Dropping — constant columns found
Patient 1: Less than 30 time points — skipping.
Patient 3: Less than 30 time points — skipping.
Patient 4: Dropping — constant columns found
Patient 5: Less than 30 time points — skipping.
Patient 9: Less than 30 time points — skipping.
Patient 11: Less than 30 time points — skipping.
Patient 12: Dropping — constant columns found
Patient 14: Less than 30 time points — skipping.
Patient 15: Less than 30 time points — skipping.
Patient 16: Dropping — constant columns found
Patient 19: Less than 30 time points — skipping.
Patient 21: Less than 30 time points — skipping.
Patient 22: Less than 30 time points — skipping.
Patient 23: Dropping — constant columns found
Patient 24: Dropping — constant columns found
Patient 25: Less than 30 time points — skipping.
Patient 26: Less than 30 time points — skipping.
Patient 31: Dropping — constant columns found
Patient 32: Dropping — constant columns found
Patient 33: Less than 30 time points — skipping.
P

In [ ]:
pa

In [5]:
y_train, y_test = loader.split_train_test(patients, test_size=4)

In [26]:
from statsmodels.tsa.stattools import adfuller

for col in y_train.columns:
    result = adfuller(y_train[col])
    
    print(f"{col}: p-value = {result[1]}")


HR: p-value = 0.004324961879061394
SBP: p-value = 0.012223742559937512
MAP: p-value = 9.253383177566546e-06
DBP: p-value = 1.578003108199769e-05
Resp: p-value = 0.003437086636660535
Temp: p-value = 9.841319187236731e-05


In [6]:
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.forecasting.var import VAR
from sktime.transformations.series.difference import Differencer

model = TransformedTargetForecaster(steps=[
    ("diff", Differencer()),     # makes the data stationary
    ("var", VAR())               # fit VAR on differenced data
])



In [7]:
from sktime.forecasting.base import ForecastingHorizon

model.fit(y_train)
fh = ForecastingHorizon([1, 2, 3, 4], is_relative=True)
forecasts = model.predict(fh)

In [8]:
from sktime.distances import dtw_distance
import numpy as np

dtw_list = []

for pid in y_test.index.get_level_values("Patient_ID").unique():
    for feature in y_test.columns:
        timeseries = y_test.loc[pid, feature]
        forecasted_timeseries = forecasts.loc[pid, feature]
        
        x = np.array(timeseries)
        y = np.array(forecasted_timeseries)
        
        dtw_val = dtw_distance(x, y)
        dtw_list.append(dtw_val)

average_dtw = np.mean(dtw_list)
print("Average DTW:", average_dtw)


Average DTW: 423.46760443300127


In [ ]:
from sktime.distances import dtwdta